In [100]:
import pandas as pd
import re
from random import randint
import spacy
from time import time

# Folketingstidende datawrangling

Det følgende script er en renskrivning af et "trial and error" script jeg tidligere har udarbejdet. Dette script kan findes på min github:

### Formål
Dette script ønsker at formatere "folketingstidende" på en sådan måde at der kan laves en digitalhistorisk analyse af data. Formålet med scriptet er da at "genkende" når en ny person begynder at tale fra folketingsets talerstol, så folektingstidende kan opdeles i folketingstaler og udtræke metadateen "hvem der taler" samt "hvilket parti tilhører taleren". Derudover skal scriptet søge at genkende når der er "fejl" i data som skal sorteres fra. Her er primært tale om at sortere marginer fra. Da der er tale om en "OCR" læsning af folketingstidende starter hver side med marginen på siden, som hvis denne ikke sorteres fra, vil indgå som en del af et folketingsmedlem eller ministers tale. 

### Genkendelse af talere:

I det følgende vil jeg søge at benytte mig af regex til at genkende når en ny person indtager folketingsets talerstol. Til dette vil jeg udnytte hvordan folketingstidende er "formateret" af udgiver. Når en ny person indtager folketingets talerstol vil dette blive skrevet som "Navn på talende folketingsmedlem (Parti som folketingsmedlem tilhører)". Det er altså dette jeg vil søge at genkende.

Jeg starter med at indlæse en del af folketingstidende som prøve.

In [127]:
df = pd.read_csv('../../Folketingstidende/Hele/19901.csv',sep=';', encoding = "latin1")

Jeg finder ny søjlenavnene:

In [128]:
df.columns.values.tolist()

['Unnamed: 0', 'spalte', 'TextBlock', 'TopMargin', 'indhold', 'år', 'samling']

Det er søjlen "indhold" der primært har interesse.

In [129]:
FT_data = [indhold for indhold in df["indhold"]]

In [130]:
FT_data[0:10]

['(1) l 2/10 90: Folketingets åbning 2',
 '1. møde',
 'Tirsdag den 2. oktober 1990 kl. 12.00',
 'Aldersformanden (Ninn-Hansen):',
 'På denne første tirsdag i oktober er Folketinget trådt sammen for at indlede folketingsåret 1990-91, og det sker som bekendt i henhold til bestemmelser i grundloven herom.',
 'Som den, der længst har været medlem af Tinget, er det min opgave og pligt at lede valget af præsidium og af tingsekretærer.',
 'Til formand har et flertal bestående af Det Konservative Folkeparti, Venstre, Fremskridtspartiet, Det Radikale Venstre, Centrum-Demokraterne og Kristeligt Folkeparti indstillet H. P. Clausen (KF).',
 'Over for mig foreligger der ikke andre indstillinger. Hvis ingen begærer afstemning, vil jeg betragte den her foretagne indstilling som vedtaget. (Ophold). Der er ingen, der begærer afstemning, og dermed er H. P. Clausen valgt.',
 'Til næstformænd har Tingets fire største partier - bortset fra det parti, som formanden tilhører - udpeget følgende medlemmer:',
 

Her viser jeg de første 10 OCR-læste afsnit i folketingstidende.

Jeg vil nu søge at genkende når der er folk der taler fra folketingsets talerstol.

In [131]:
Prøve_FT = []
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)", "(KRF)"] #liste af partier
for sentence in FT_data:
    #if len(re.findall("(\w{1,10}\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0:
        if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
            if len(re.findall("Ny stedf.", sentence)) > 0:
                pass
            elif len(re.findall("Udtræder:", sentence)) > 0:
                pass
            elif len(re.findall("Nyt medl", sentence)) > 0:
                pass
            elif len(re.findall(" af", sentence.lower())) > 0:
                pass
            elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                Prøve_FT.append(sentence)

Jeg kan nu undersøges hvad der genkendes:

In [132]:
Prøve_FT.sort(key = len, reverse = True)

In [133]:
len(Prøve_FT)

2436

In [134]:
Prøve_FT[0:100]

['forkastedes, idet 13 (SF) stemte for, 56 (KF, V, FP, RV og CD) imod; 34 (S og KRF) stemte hverken for eller imod.',
 've Niemann (FP) har meddelt mig, at de til justitsministeren ønsker at stille følgende:',
 'Af Birgitte Husmark (SF) m.fl. (Forespørgslen anmeldt 4/10 90).',
 'Af Kirsten Jacobsen (FP) m.fl. (Forespørgslen anmeldt 8/11 90).',
 'Af Margrete Auken (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 'Af Tommy Dinesen (SF) m.fl. (Forespørgslen anmeldt 10/10 90).',
 'Af Pia Kjærsgaard (FP) m.fl. (Forespørgslen anmeldt 8/11 90).',
 'Af Aage Frandsen (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 '15 (SF) stemte hverken for eller imod.',
 'Af Rahbæk Møller (SF), Dorte Bennedsen',
 'Af Rahbæk Møller (SF), Dorte Bennedsen',
 'Nyt med},: Else Winther Andersen (V)',
 '16-17 Hanne Thanning Jacobsen (SF)',
 '16-17 Hanne Thanning Jacobsen (SF)',
 "1-3 Pia Christmas-Møller (KF) '.I",
 'Birgitte Husmark (SF) (se foran).',
 '17 Hanne Thanning Jacobsen (SF)',
 '17 Hanne Thanning Jacobsen (S

Det ovenstående søger at genkende når der er et menigt medlem(hermed ment ikke formanden eller en minister) som ønsker at tale. Problemet er her at opremsningen af medlemmer af folketinget i starten af folketingsåret genkendes som "en taler" samt at metatale lavet af formanden genkendes eks. "Medlem af folketinget Askjær Jørgensen (FP) har meddelt mig, at han ønsker til industriministeren at stille følgende:". I begge tilfælde ser jeg det dog som mindre problematisk. 
I første tilfælde vil de "genkendte" talere blive tildelt en "tale" som er tom, altså at disse ikke siger noget, hvilket vil blive sorteret fra når jeg udtrækker de folketingstaler jeg ønsker at analysere. 
I andet tilfælde af problemet er ligeledes mindre problematisk. Her er der tale om at folketingets formand fremsætter et menigt medlems holdninger, og deraf ser jeg det ikke problematisk at "karakterisere" disse som folketingstaler.

### Genkendelse af ministre:

In [144]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [136]:
ministre_FT.sort(key = len, reverse = True)

In [145]:
ministre_FT[0:1000]

['Statsministeren (Poul Schlüter):',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Skatteministeren (Fogh Rasmussen) har meddelt mig, at han ønsker skriftligt at fremsætte :',
 'Finansministeren (Henning Dyremose):',
 'Skatteministeren (Fogh Rasmussen):',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Indenrigsministeren (Thor Pedersen) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Kirkeministeren (Torben Rechendorff) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte :',
 'Justitsministeren (Engell) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Skatteministeren (Fogh Rasmussen) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Socialministeren (Aase Olesen) har meddelt mig, at hun ønsker skriftligt at fremsætte:',
 'Socialministeren (Aase Olesen) h

In [142]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\([\w-]*\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [143]:
ministre_FT

['Justitsministeren (Engell) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Justitsministeren (Engell):',
 'Landbrugsministeren (Tørnæs) har meddelt mig, at han ønsker skriftligt at fremætte:',
 'Landbrugsministeren (Tørnæs):',
 'Af justitsministeren (Engell).',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Af justitsministeren (Engell).',
 'Justitsministeren (Engell):',
 'Forslag til lov om frø, kartofler og planter. Af landbrugsministeren (Tørnæs).',
 'Justitsministeren (Engell) har meddelt mig, at han ønsker skriftligt af fremsætte:',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',

Det ovenstående genkender meget godt når der er tale om en minister.

In [119]:
p = "Energiministeren (Bilgrav-Nielsen)"
len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", p.lower()))

1

### Genkendelse af Formanden:

In [80]:
count_formanden = 0
for sentence in FT_data:
    if sentence.lower() == "formanden" or sentence.lower() == "formanden:":
        count_formanden += 1
count_formanden

1024

### Genkendelse af næstformand

In [82]:
n = 0
for sentence in FT_data:
    if re.search("næstformand \(\w*\):$", sentence.lower()) or re.search("næstformand \(\w* \w*\):$", sentence.lower()):
        n += 1
        
n

448

Jeg undersøger nu om der kan være nogle datapunkter der er blevet misset:

In [81]:
formanden_problem = []
for sentence in FT_data:
    if len(sentence) <= 15:
        if len(re.findall("formanden", sentence.lower())) > 0:
            if sentence.lower() != "formanden" and sentence.lower() != "formanden:":
                formanden_problem.append(sentence)
formanden_problem[0:10]

['[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]']

I det ovenstående er der tale om at der står i margenen af folketingstidende, at det er formanden der taler, det er derfor ikke problematisk at disse sorteres fra.

### Marginer sorteres fra:

In [82]:
hele_margin = []
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
        if len(sentence) < 200:
            hele_margin.append(sentence)
print(len(hele_margin))
hele_margin[0:10]

2517


['11 7/10 80: Statsministerens åbningstale 12',
 '15 7/10 80: Statsministerens åbningstale 16',
 '17 7/10 80: Statsministerens åbningstale IR',
 '19 7/10 80: Statsministerens åbningstale 20',
 '21 7/10 80: Statsministerens åbningstale. Skr. del 22',
 '23 7/10 80: Statsministerens åbningstale. Skr. del 24',
 '35 7/10 80: Statsministerens åbningstale. Skr. del 36',
 '39 7/10 80: Statsministerens åbningstale. Skr. del 40',
 '41 7/10 80: Statsministerens åbningstale. Skr. del 11 42',
 '43 7/10 80: Statsministerens åbningstale. Skr. del 44']

In [96]:
hele_margin_prøve = []
for sentence in FT_data:
    if len(sentence) <= 170: 
        if len(re.findall("[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower()))  > 0:
            hele_margin_prøve.append(sentence)
print(len(hele_margin_prøve))
hele_margin_prøve[0:10]

3842


['11 7/10 80: Statsministerens åbningstale 12',
 '15 7/10 80: Statsministerens åbningstale 16',
 '17 7/10 80: Statsministerens åbningstale IR',
 '19 7/10 80: Statsministerens åbningstale 20',
 '21 7/10 80: Statsministerens åbningstale. Skr. del 22',
 '23 7/10 80: Statsministerens åbningstale. Skr. del 24',
 '(3) 33 7/10 80: Statsministerens åbningstale. Skr. del 34',
 '35 7/10 80: Statsministerens åbningstale. Skr. del 36',
 '39 7/10 80: Statsministerens åbningstale. Skr. del 40',
 '41 7/10 80: Statsministerens åbningstale. Skr. del 11 42']

In [98]:
hele_margin_prøve.sort(key = len, reverse = True)
hele_margin_prøve[0:10]

['(587) 9377 3/4 81: l. beh. af f. t. beslutn. vedr. tilbagetrækning af aktstykkerne nr. 285, 286, 287 og 292 9378',
 '(785) 12545 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12546',
 '(787) 12577 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12578',
 'I 8175 18/3 81: l. beh. af f. t. beslutn. om forenkling af erhvervslivets indbetalinger til det offentlige 8176',
 '(755) 12065 26/5 81: 2. (sidste) beh. af f. t. beslutn. vedr. latinprøve på 9. klassetrin i ungdomsskolen 12066',
 '8173 18/3 81: l. beh. af f. t. beslutn. om forenkling af erhvervslivets indbetalinger til det offentlige 8174',
 '8177 18/3 81: 1. beh. af f. t. beslutn. om forenkling af erhvervslivets indbetalinger til det offentlige 8178',
 '8181 18/3 81: l. beh. af f. t. beslutn. om forenkling af erhvervslivets indbetalinger til det offentlige 8182',
 '8183 18/3 81: l. beh. af f. t. beslutn. om forenkling af erhvervslivets i

Det er klart at det ovenstående er marginerne på hver side. Hvis man dog følger tallet til venstre, som er det første spalte(af to) på hver side af folketingstidende, kan man dog se at det ikke er alle marginer der findes her.

In [97]:
dato_alene = []
for sentence in FT_data:
    if len(re.findall("[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                dato_alene.append(sentence)
print(len(dato_alene))
dato_alene[0:10]

9245


['7/10 80: Folketingets åbning',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '11 7/10 80: Statsministerens åbningstale 12',
 '7/10 80: Statsministerens åbningstale',
 '15 7/10 80: Statsministerens åbningstale 16',
 '17 7/10 80: Statsministerens åbningstale IR',
 '19 7/10 80: Statsministerens åbningstale 20']

Her kan det også ses at der er tale om marginer. Disse skal derfor ligeledes sorteres fra. Problemet er her at spaltenumrene da er sorteret som individuelle datapunkter. Jeg prøver da også at sortere disse fra.

In [84]:
tal_genkendt = []
for sentence in FT_data:
    if re.fullmatch("[1-9]{1,10}", sentence.lower()): #tilføjet til at fange headers på sider
        tal_genkendt.append(sentence)
tal_genkendt[0:10]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '13']

Hvis man kigger tallene igennem er det meget plausibelt at der her er tale om de manglende spaltenumre. Disse skal altså også sorteres fra.

I marginerne optræder altså også en navn på vedkommende der er den "tallende" når der skiftes spalte. Disse vil jeg også forsøge at sortere fra, da jeg ikke ønsker at en taler fra folketingets talerstols eget navn skal indgå i dennes tale. Heldigvis har disse navne også en relativt nem formatering, nemlig \[navn\]. Jeg søger altså efter datapunker der matcher denne formatering.

In [85]:
tallere_fra_forrige_side = []
for sentence in FT_data:
    if re.fullmatch("\[.{1,100}\]", sentence.lower()):
        tallere_fra_forrige_side.append(sentence)
print(len(tallere_fra_forrige_side))

6039


In [86]:
tallere_fra_forrige_side[0:10]

['[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]']

### Tallere fra forrige side som ikke er blevet læst som seperate enheder:

In [87]:
tallere_fra_forrige_side_læst_forkert = []
for sentence in FT_data:
    if not re.fullmatch("\[.{1,100}\]", sentence.lower()):
        if not re.fullmatch("\[.{1,100}\].", sentence.lower()):
            if re.match("^\[.{1,100}\]", sentence.lower()):
                tallere_fra_forrige_side_læst_forkert.append(sentence)
print(len(tallere_fra_forrige_side_læst_forkert))

7


In [88]:
tallere_fra_forrige_side_læst_forkert

['[Vedr. stedfortræder for Arentoft]. (Beslutningsforslag nr. B 42).',
 '[Energiministeren] x I om Ekofiskfeltet i 1965, som hr. Svend Heiselberg her gør det.',
 '[Ministeren for offentlige arbejder] indvandsreservoir, og ministeriet for offentlige arbejder har i overensstemmelse hermed den principielle opfattelse, at etablering af et saltvandsindtag derfor kun bør ske på grundlag af en særlig lov.',
 "[Dyremose] '--- vil have ministerens forklaring på, hvorfor man selv i en sag som denne, der tilsyneladende er lille, giver folketinget en meget mangelfuld og derfor i en vis udstrækning fejlagtig information, og hvorfor man så, når man har begået en fejl i foråret, skjuler dette ved i bemærkningerne til det aktuelle lovforslag blot at anføre, at ministeriet for skatter og afgifter har oplyst, at der er behov for en ændring, uden at fortælle, at det i virkeligheden er, fordi man har begået en fejl i regeringen.",
 '[Undervisningsministeren] køkkenassistentuddannelsen, nemlig mod arbejde 

Her findes de forskellige navne fra marginerne, som skal sorteres fra.

## Sammensætning af det ovenstående

I det følgende vil jeg sætte alt det ovenstående sammen, med det formål at udtrække folketingstaler, samt hvem der taler, og sortere marginer fra.

In [79]:
indhold = FT_data
talere = []
tale = []
parti = []
count_folketing = 0
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif re.search("næstformand \(\w*\)", sentence.lower()) or re.search("næstformand \(\w* \w*\)", sentence.lower()):
            talere.append(sentence.lower())
            parti.append("næstformanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence

Jeg laver nu en funktion ud af dette:

In [121]:
def FT_taler(indhold):
    talere = []
    tale = []
    parti = []
    count_folketing = 0
    Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
    for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif re.search("næstformand \(\w*\):$", sentence.lower()) or re.search("næstformand \(\w* \w*\):$", sentence.lower()):
            talere.append(sentence.lower())
            parti.append("næstformanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence
    return tale, talere, parti

Ovenstående funktion danner basis for formatering af data til senere analyse.